Management won't be random. Cultivars will be caracterized according to their maturity tipe and their yield potential based on DSSAT parameters. q parameter is removed from the calibration. All high yield counties.

In [1]:
import sys
sys.path.append("/home/dquintero/spatialDSSAT")
from spatialDSSAT.run import GSRun
sys.path.append("/home/dquintero/venvs/serviceDSSAT/lib/python3.10/site-packages")
sys.path.append("/home/dquintero/dssat_service/regional_service")
from dssat import run_spatial_dssat

In [2]:
import pickle
import numpy as np
import pandas as pd

from scipy import stats

import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt

from datetime import datetime, timedelta
from multiprocessing import Pool
from itertools import product
import time
import os
from tqdm.notebook import tqdm

In [9]:
CULTIVARS = [
    'IF0001', 'IF0002', 'IF0003', 'IF0004', 'IF0005', 'IF0006', 'IF0007', 
    'IF0008', 'IF0009', 'IF0010', 'IF0011', 'IM0001', 'IM0002', 'IM0003', 
    'IF0018', 'IF0019', 'IF0020', 'IF0021', 'IF0022', 'IB0067', 'KA0001', 
    'EM0001', 'KY0001', 'KY0002', 'KY0003', 'KY0004', 'KY0005', 'KY0006', 
    'KY0007', 'KY0008', 'KY0009', 'KY0010', 'KY0011', 'KY0012', 'KY0013', 
    'KY0014', 'KY0015', 'KY0016', 'KY0017', 'KY0018', 'GH0010'
]
ADMIN1_LIST = [
    'Trans Nzoia', 'Bungoma', 'Uasin Gishu', 'Nandi', 'Kakamega', 'Kisii','Nakuru', 'Siaya', 'Migori', 
    'Narok', 'Nyamira', 'Meru', 'Kericho','Bomet',
    'Makueni'
]
DBNAME = "dssatserv"
obs = pd.read_csv("~/dssat_service/fewsnet_data/kenya_shortRains_maize.csv")
obs = obs.loc[obs.year > 2010]

In [18]:
def get_dssat_inputs(admin1, year):
    inputs = run_spatial_dssat(
        dbname=DBNAME, 
        schema="kenya", 
        admin1=admin1,
        plantingdate=datetime(year, 3, 1),
        cultivar=CULTIVARS[0],
        nitrogen=[(5, 15), (30, 15), (60, 15)],
        # overview=True,
        all_random=False,
        return_input=True
    )
    return inputs

In [16]:
# Get inputs for all years
admin1 = ADMIN1_LIST[1]
inputs_dict = {}
for _, row, in obs.loc[obs.admin_1 == admin1].iterrows():
    inputs_dict[row.year] = get_dssat_inputs(admin1, row.year)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 17.34it/s]


In [41]:
def run_single(admin1, fertilizer, cultivar, plantingdate, inputs):
    fertilizer = int(fertilizer)
    fertilizer = int(fertilizer/3)
    nitro = [(0, fertilizer), (30, fertilizer), (60, fertilizer)]
    gs = GSRun()
    for (_, wth), (_, sol) in inputs:
        gs.add_treatment(
            soil_profile=sol,
            weather=wth,
            nitrogen=nitro,
            planting=plantingdate,
            cultivar=cultivar
        )
    planting_window_start = plantingdate - timedelta(days=15)
    planting_window_end = plantingdate + timedelta(days=15)
    sim_controls = {
        "PLANT": "F", # Automatic, force in last day of window
        "PFRST": planting_window_start.strftime("%y%j"),
        "PLAST": planting_window_end.strftime("%y%j"),
        "PH2OL": 50, "PH2OU": 100, "PH2OD": 20, 
        "PSTMX": 40, "PSTMN": 10
    }
    df = gs.run(
        start_date=plantingdate - timedelta(30),
        sim_controls=sim_controls
    )
    return df

In [42]:
def calculate_mse(admin1, fertilizer, cultivar):
    residuals = []
    for _, row in obs.loc[obs.admin_1 == admin1].iterrows():
        plantingdate = datetime(row.year, 4, 1)
        df = run_single(admin1, fertilizer, cultivar, plantingdate, inputs_dict[row.year])
        residuals.append(df.HARWT.astype(int).median()/1000 - row.value)
    return sum([r**2 for r in residuals])

In [43]:
calculate_mse(admin1, 50, CULTIVARS[0])

6.13453416528559

In [61]:
from scipy.optimize import minimize_scalar
def optimize_nitrogen(admin1, cultivar):
    res = minimize_scalar(
        fun=lambda x: calculate_mse(admin1, x, cultivar),
        bounds=(0, 200),
        options={"maxiter": 10, "disp": 3, "xatol": 5}
    )
    return res.x

In [62]:
cultivar = CULTIVARS[0]
nitro = optimize_nitrogen(admin1, cultivar)

 
 Func-count     x          f(x)          Procedure
    1        76.3932      39.6086        initial
    2        123.607       148.59        golden
    3        47.2136      4.61162        golden
    4        20.4905      9.35478        parabolic
    5        37.4553      3.06314        parabolic
    6        39.1219      3.12046        parabolic
    7        35.7886      3.33453        parabolic

Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  5 )


In [63]:
nitro

37.455280335329675

In [69]:
from scipy.stats import pearsonr
records = []
# admin1 = ADMIN1_LIST[1]
for cultivar in CULTIVARS[:5]:
    nitro = optimize_nitrogen(admin1, cultivar)
    for month in range(3, 6):
        sim_yield = []
        for _, row in obs.loc[obs.admin_1 == admin1].iterrows():
            plantingdate = datetime(row.year, month, 1)
            df = run_single(admin1, nitro, cultivar, plantingdate, inputs_dict[row.year])
            sim_yield.append(df.HARWT.astype(int).median()/1000)
        obs_yield = obs.loc[obs.admin_1 == admin1, "value"].values
        r, p =pearsonr(sim_yield, obs_yield)
        me = (np.array(sim_yield) - obs_yield).mean()
        mae = abs(obs_yield - np.array(sim_yield)).mean()
        rmse = np.sqrt((obs_yield - np.array(sim_yield))**2).mean()
        records.append((
            admin1, cultivar, nitro, month, r, p, me, mae, rmse
        ))
        print(records[-1])

 
 Func-count     x          f(x)          Procedure
    1        76.3932      39.6086        initial
    2        123.607       148.59        golden
    3        47.2136      4.61162        golden
    4        20.4905      9.35478        parabolic
    5        37.4553      3.06314        parabolic
    6        39.1219      3.12046        parabolic
    7        35.7886      3.33453        parabolic

Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  5 )
('Bungoma', 'IF0001', 37.455280335329675, 3, 0.026667720265396024, 0.9547435684403146, -0.42386791428571424, 0.821679162857143, 0.821679162857143)
('Bungoma', 'IF0001', 37.455280335329675, 4, -0.3752883108881727, 0.4067920005779302, -0.10401077142857147, 0.46965152857142856, 0.46965152857142856)
('Bungoma', 'IF0001', 37.455280335329675, 5, -0.06872174432856945, 0.8836096352914243, 0.1808463714285715, 0.5192505914285713, 0.5192505914285713)
 
 Func-count     x          f(x)         

In [83]:
print("{0}\t{1}\t{2:>.1f}\t{3:>}\t{4:>.1f}\t{5:>.3f}\t{6:>.3f}\t{7:>.3f}\t{8:>.3f}".format(*records[-1]))

Bungoma	IF0005	44.6	5	0.3	0.465	0.313	0.519	0.519


In [24]:
wth

'/tmp/tmpvatufzz4/WS262001.WTH'

In [9]:
len(inputs)

27

In [14]:
inputs[0]

(((34.958, 0.792), '/tmp/tmpsqlpzxm0/WS002001.WTH'),
 ((34.958, 0.792),
  '*KE04624979    KEN   SandClayL   200    ISRIC soilgrids + HC27                                                                                                                                          \n@SITE        COUNTRY          LAT     LONG SCS Family                                                                                                                                                   \n -99              KE        0.792   34.958     HC_GEN0011                                                                                                                                               \n@ SCOM  SALB  SLU1  SLDR  SLRO  SLNF  SLPF  SMHB  SMPX  SMKE                                                                                                                                            \n    BK  0.10  6.00  0.50 75.00  1.00  1.00 SA001 SA001 SA001                                                         